In [1]:
## lm studioのAPIが生きてるか確認
import requests, pprint
pprint.pp(requests.get("http://localhost:1234/v1/models").json())

{'data': [{'id': 'llama-3-elyza-jp-8b',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'text-embedding-nomic-embed-text-v1.5',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'deepseek-r1-distill-llama-8b',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'llm-jp-3-13b-instruct',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'deepseek-r1-distill-qwen-7b',
           'object': 'model',
           'owned_by': 'organization_owner'},
          {'id': 'llama-3.2-1b-instruct',
           'object': 'model',
           'owned_by': 'organization_owner'}],
 'object': 'list'}


In [2]:
from openai import OpenAI

model_name = 'llama-3-elyza-jp-8b'

client = OpenAI(
    base_url="http://localhost:1234/v1",  # ← ポートを変えた場合はここも変える
    api_key="lm-studio"                  # 何でも良い
)

response = client.chat.completions.create(
    model=model_name,  # LM Studio に表示される識別子
    messages=[
        {"role": "system", "content": "あなたはIQ1000の超天才科学者です。どんな質問にも答えます。"},
        {"role": "user",   "content": "2050年の民主主義はどうなってるか教えてください"}
    ],
    temperature=0.7,
    stream=False,        # True にすればストリームで chunk が取れる
)
print(response.choices[0].message.content)

2050年の民主主義という未来予測を行うのは非常に困難で、多くの仮定や前提が必要です。しかしながら、現状の趨勢と技術の進化を考慮すると、以下は一つの可能性として考えられます。

2050年には、人工知能(AI)、ブロックチェーン、バイオテクノロジーなどの技術が更に進歩し、民主主義も変革することが予測されます。例えば:

1. 遠隔投票や電子投票の普及により、選挙プロセスはより効率的、公正かつ透明性を高めることが可能です。
2. AIとブロックチェーン技術の組み合わせで、選挙結果の自動監査や不正防止が強化されます。
3. デジタル・民主主義の進展に伴い、有権者はオンライン上で情報を得ることが容易になり、政治家とのコミュニケーション方法も多様化します。
4. バイオテクノロジーの進化により、生体認証やDNA鑑定が選挙プロセスに組み込まれ、有権者確認や投票の真正性を担保することができます。
5. 環境問題や経済的格差など、複雑な社会課題に対応するために、多元的、分散型の民主主義が模索されます。例えば、AIによる意思決定支援や、住民投票の活用などです。

2050年の民主主義は、テクノロジーと政治のより緊密な結びつきを特徴とし、「デジタル・民主主義」あるいは「超民主主義」と呼ばれる新しい形態に進化する可能性があります。


In [3]:
# MEMO: LM Studioはjsonフォーマットでの出力を強制できないため、生成とパースが容易なXMLで出力させる
from openai import OpenAI
import re

def generate_variation_response(text):
    model_name = 'llama-3-elyza-jp-8b'

    client = OpenAI(
        base_url="http://localhost:1234/v1",  # ← ポートを変えた場合はここも変える
        api_key="lm-studio"                  # 何でも良い
    )

    system_prompt = """あなたは入力されたテキストを改変し、**20** 個 のバリアントを作成します。
バリアントは、意味を大きく変えないように注意しながら、次の戦略で作りなさい。
- 一つ前のバリアントの文章を参照して改変する
- 名詞や動詞を類義語や別の言葉に置き換える
- 単語を抽象化したり、具体化したりする
- 述語の順序を入れ替える、文章の順序を入れ替える
- 文章を短くする、長くする
- 新たな言葉を追加する
- 意味が通る日本語の文章にする、英文にしないこと

出力は必ず次に示すXML形式で返してください。

```xml
<original>{original_text}</original>
<variant_01>{variant_text_01}</variant_01>
<variant_02>{variant_text_02}</variant_02>
...省略
<variant_19>{variant_text_19}</variant_19>
<variant_20>{variant_text_20}</variant_20>
```
"""

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": text}
        ],
        temperature=1.2,
        presence_penalty=1.0,       # 同じトークンの出力の抑制
        frequency_penalty=1.0,      # 同じトークンが何度も出た場合の追加ペナルティ
        stream=False,
    )

    # print(pprint.pformat(response.choices[0].message.content))
    # 生成されたXMLからバリエーションを抽出する
    variants = re.findall(r"<\s*variant_\d+\s*>(.*?)</\s*variant_\d+\s*>", response.choices[0].message.content, re.DOTALL)
    return variants

target_text = "自然災害、特に地震が多い日本で原子力発電所を新設・再稼働をすすめることは、大きな危険を長期にわたり国民に背負わせることとなります。"
variants = generate_variation_response(target_text)
# print(target_text)
for v in variants:
    print(v)

# 重複を確認する
print(len(variants), len(set(variants)))


大規模な自然災害が相次ぐ日本において、原子力発電所の新設や再稼働を推進する動きは、深刻なリスクを長期的に国民に押し付けていると言えます。
多くの地震被害を受ける日本で原子力発電所を新たに建設したり運転再開の道筋をつけたりすることは、甚大な危険性が長きにわたり国民生活に影を落とすことになるでしょう。
地震や津波等の自然災害が頻発している日本で原子力発電所の新設・再稼働推進は、巨大な不安要素を長期化する結果として国民に重くのしかかることになります。
3 3


In [ ]:
variant_list = []

target_text = "自然災害、特に地震が多い日本で原子力発電所を新設・再稼働をすすめることは、大きな危険を長期にわたり国民に背負わせることとなります。"
variant_list.append(target_text)
for i in range(10):
    variants = generate_variation_response(target_text)
    for v in variants:
        if v in variant_list:
            continue
        
        # アルファベットが3割以上のものは除外する、英語の文章が混じっている可能性があるため
        if len(re.findall(r"[a-zA-Z]", v)) / len(v) >= 0.3:
            continue

        print(v)
        variant_list.append(v)

# export file
with open("variant_list.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(variant_list))